In [65]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import serial
import tdwf
import time
import os

#-[Configurazione seriale]-----------------------------------------------------
ser = serial.Serial()
ser.port = "COM7"
ser.baudrate = 115200
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n'
        ser.write(message.encode('utf-8'))
    if flagR:
        res = ser.readline().decode('utf-8')
        return res


for i in range(12):
    response = ser.readline().decode('utf-8')
    print(response[:-1])


#-[Configurazione AD2]-------------------------------------------------------
ad2 = tdwf.AD2(iconfig=2)

ad2.vdd = +5
ad2.power(True)

#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.config(ampl = 5, offs= 0, func=tdwf.funcSine, freq = 40968)
wgen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=1e6
scope.npt=int(16384)
scope.ch1.rng = 10
scope.ch2.rng = 10
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1, delay = 0.04)


#-[Saving pipeline]-----------------------------------------------------
Directory = f"angular_{time.strftime('%Y%m%d_%H%M%S')}" # directory name based on the current date and time
parent_dir = "../../Data"
path = os.path.join(parent_dir, Directory) # creating the directory
os.mkdir(path)
print(f"Data will be saved in {path}\n")
log_file = os.path.join(path, f"log.txt")   #creating the log file
message = input("Please insert a \bMEANINGFUL\b comment to understand what the measurement in about: ")
with open(path+"/log.txt", "a") as log:
    log.write(time.strftime('%H%M%S') + ' >>> ' + message + "\n")
    log.close()





JoyStepper: XYZ stepper motor controller with joystick
 Interrupt frequency (real):  25000 Hz
 Interrupt frequency (ideal): 25000 Hz
 Update frequency (real):     1000 Hz
 Update frequency (ideal):    1000 Hz
------------------------------------------------------
Speed X 4000 msps
Speed Y 4000 msps
Speed Z 600 msps
Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #2
Data will be saved in ../../Data\angular_20250515_100623



In [66]:
import numpy as np
rangeX = [0, 80000]
rangeY = [-400, 400]



translations = np.linspace(rangeX[0], rangeX[1], 11, dtype=int)
angules = np.linspace(rangeY[0], rangeY[1], 11, dtype=int)


#print(angules)



def setAngle(angle):
    print( query("ty="+str(angle))[:-1] )
    res = int(query("y?").split('=')[1])
    while res != angle:
        res = int(query("y?").split('=')[1])
    return res

def setX(x):
    print( query("tx="+str(x))[:-1] )
    res = int(query("x?").split('=')[1])
    while res != x:
        res = int(query("x?").split('=')[1])
    return res


setX(0)
setAngle(0)


for i in range(len(angules)):
    setAngle(angules[i])

#    res = query("ty="+str(angules[i]))
    # print(res[:-1])

    # make a measurement at the current angle in order to get the angular power distribution
    scope.sample()
    tt = scope.time.vals.tolist()
    tx = scope.ch1.vals.tolist()
    rx = scope.ch2.vals.tolist()

    info = f"Angular measurement - Y = {angules[i]:3.2f} \n"
    data_to_save = np.c_[tt, tx, rx]
    info = info + f"time [s]\tch1 [V]\tch2 [V]\n"

    filename = f"angular_{angules[i]:.0f}"
    np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")


break














res = query("tx="+str(rangeX[1]))
print(res[:-1])

posX = int(query("x?").split('=')[1])
first = True
time.sleep(.5)


while(posX < 0.98 * rangeX[1]):
    posX = int(query("x?").split('=')[1])
    print(posX)
scope.sample()
#    tt.append(scope.time.vals.tolist())
#    tx.append(scope.ch1.vals.tolist())
#    rx.append(scope.ch2.vals.tolist())


tt = tt + scope.time.vals.tolist()
tx = tx + scope.ch1.vals.tolist()
rx = rx + scope.ch2.vals.tolist()
res = query("tx=" + str(rangeX[0]))
print(res[:-1])

info = f"Doppler Shift measurement - X = {rangeX[0]:5.0f} -> {rangeX[1]:5.0f} - vX = {query("sx?").split('=')[1] } \n"

data_to_save = np.c_[tt, tx, rx]
info = info + f"time [s]\tch1 [V]\tch2 [V]\n"


filename = f"dopplerShift-{rangeX[0]:.0f}-{rangeX[1]:.0f}"

np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")



TX=0
TY=0
TY=-400
TY=-320
TY=-240
TY=-160
TY=-80
TY=0
TY=80
TY=160
TY=240
TY=320
TY=400


SyntaxError: 'break' outside loop (596638296.py, line 54)

In [68]:
query(input(">> "))

'TY=0\r\n'

In [64]:
ser.close()